In [94]:
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

from keras.preprocessing.image import ImageDataGenerator
from keras.utils import load_img
from keras.utils import img_to_array
from keras.applications import imagenet_utils
import os

import pandas as pd
import tensorflow_datasets as tfds
import tensorflow_model_optimization as tfmot

In [108]:
model = tf.keras.applications.MobileNet(weights='imagenet', input_shape=(224, 224, 3),
                                             )  #include_preprocessing=False)

In [109]:
model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                                 
 conv_dw_1_bn (BatchNormaliz  (None, 112, 112, 32)     128       
 ation)                                         

In [110]:
def prepare_image(file):
    img_path = 'data/images/'
    img = load_img(img_path + file, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [65]:
def process_image(data):
    data['image'] = (tf.image.resize(data['image'], (224, 224)) * 2.0 / 255.0) - 1.0
    return data

In [6]:
# Representative dataset
def representative_dataset(dataset):
    def _data_gen():
        for data in dataset.batch(1):
            yield [data['image']]

    return _data_gen

In [89]:
def eval_tflite(tflite_model, dataset):
    """Evaluates tensorflow lite classification model with the given dataset."""
    interpreter = tf.lite.Interpreter(model_content=tflite_model)
    interpreter.allocate_tensors()

    input_idx = interpreter.get_input_details()[0]['index']
    output_idx = interpreter.get_output_details()[0]['index']

    results = []

    total = 0
    for data in representative_dataset(dataset)():
        total = total + 1
        if total % 10 == 0:
            print("Evaluated", total)
        interpreter.set_tensor(input_idx, data[0])
        interpreter.invoke()
        results.append(interpreter.get_tensor(output_idx).flatten())

    results = np.array(results)
    gt_labels = np.array(list(dataset.map(lambda data: data['label'])))
    accuracy = (
            np.sum(np.argsort(results, axis=1)[:, -1:] == gt_labels.reshape(-1, 1)) /
            gt_labels.size)
    print(f'Top-1 accuracy (quantized): {accuracy * 100:.2f}%')

In [111]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [127]:
tr_ds = tfds.load('imagenet_v2', split='test[:50%]')
tr_ds = tr_ds.map(process_image)

train_ds = tr_ds\
    .map(lambda data: (data['image'], data['label']))\
    .batch(32)

ds = tfds.load('imagenet_v2', split='test[90%:]')
ds = ds.map(process_image)

In [129]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.representative_dataset = representative_dataset(tr_ds)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
quantized_model = converter.convert()

INFO:tensorflow:Assets written to: /var/folders/jt/1gfgpp697fv8mf_x_j2bdcm80000gn/T/tmppgs4z6pe/assets


INFO:tensorflow:Assets written to: /var/folders/jt/1gfgpp697fv8mf_x_j2bdcm80000gn/T/tmppgs4z6pe/assets
/Users/miroslavsafar/opt/anaconda3/envs/bachelor_thesis/lib/python3.10/site-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-01-23 16:42:52.135646: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-01-23 16:42:52.135663: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-01-23 16:42:52.135779: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/jt/1gfgpp697fv8mf_x_j2bdcm80000gn/T/tmppgs4z6pe
2023-01-23 16:42:52.151044: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2023-01-23 16:42:52.151075: I tensorflow/cc/saved_model/reader.cc:122] Reading

In [118]:
test_ds = ds.map(lambda data: (data['image'], data['label'])).batch(64)
loss, acc = model.evaluate(test_ds)
print(f'Top-1 accuracy (float): {acc * 100:.2f}%')

16/16 [==============================] - 26s 2s/step - loss: 2.1468 - accuracy: 0.5450
Top-1 accuracy (float): 54.50%


In [130]:
eval_tflite(quantized_model, ds)

INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


Evaluated 10
Evaluated 20
Evaluated 30
Evaluated 40
Evaluated 50
Evaluated 60
Evaluated 70
Evaluated 80
Evaluated 90
Evaluated 100
Evaluated 110
Evaluated 120
Evaluated 130
Evaluated 140
Evaluated 150
Evaluated 160
Evaluated 170
Evaluated 180
Evaluated 190
Evaluated 200
Evaluated 210
Evaluated 220
Evaluated 230
Evaluated 240
Evaluated 250
Evaluated 260
Evaluated 270
Evaluated 280
Evaluated 290
Evaluated 300
Evaluated 310
Evaluated 320
Evaluated 330
Evaluated 340
Evaluated 350
Evaluated 360
Evaluated 370
Evaluated 380
Evaluated 390
Evaluated 400
Evaluated 410
Evaluated 420
Evaluated 430
Evaluated 440
Evaluated 450
Evaluated 460
Evaluated 470
Evaluated 480
Evaluated 490
Evaluated 500
Evaluated 510
Evaluated 520
Evaluated 530
Evaluated 540
Evaluated 550
Evaluated 560
Evaluated 570
Evaluated 580
Evaluated 590
Evaluated 600
Evaluated 610
Evaluated 620
Evaluated 630
Evaluated 640
Evaluated 650
Evaluated 660
Evaluated 670
Evaluated 680
Evaluated 690
Evaluated 700
Evaluated 710
Evaluated 720
E

In [113]:
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
                      loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                      metrics=['accuracy'])

q_aware_model.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 quantize_layer_1 (QuantizeL  (None, 224, 224, 3)      3         
 ayer)                                                           
                                                                 
 quant_conv1 (QuantizeWrappe  (None, 112, 112, 32)     929       
 rV2)                                                            
                                                                 
 quant_conv1_bn (QuantizeWra  (None, 112, 112, 32)     129       
 pperV2)                                                         
                                                                 
 quant_conv1_relu (QuantizeW  (None, 112, 112, 32)     3         
 rapperV2)                                      

In [119]:
q_aware_model.fit(train_ds, epochs=3, validation_data=test_ds)

Epoch 1/3
157/157 [==============================] - 675s 4s/step - loss: 4.4786 - accuracy: 0.1730 - val_loss: 7.0806 - val_accuracy: 0.0320
Epoch 2/3
157/157 [==============================] - 712s 5s/step - loss: 1.4518 - accuracy: 0.6682 - val_loss: 6.7671 - val_accuracy: 0.0710
Epoch 3/3
157/157 [==============================] - 685s 4s/step - loss: 0.4757 - accuracy: 0.8950 - val_loss: 6.0358 - val_accuracy: 0.1200


In [115]:
qa_loss, qa_acc = q_aware_model.evaluate(test_ds)
print(f'Top-1 accuracy (quantize aware float): {qa_acc * 100:.2f}%')

16/16 [==============================] - 33s 2s/step - loss: 11.3840 - accuracy: 0.0000e+00
Top-1 accuracy (quantize aware float): 0.00%
